In [1]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from pickle import dump, load

data = pd.read_csv("../input/spaceship-titanic/train.csv")

min_max_scaler = MinMaxScaler()
min_max_scaler.fit(data[["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]])
data[["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]] = min_max_scaler.transform(data[["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]])   

dump(min_max_scaler, open('min_max_scaler.pkl', 'wb'))

In [2]:
data.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,0.493671,False,0.000000,0.000000,0.000000,0.000000,0.000000,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,0.303797,False,0.007608,0.000302,0.001064,0.024500,0.001823,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,0.734177,True,0.003001,0.119948,0.000000,0.299670,0.002030,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,0.417722,False,0.000000,0.043035,0.015793,0.148563,0.007997,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,0.202532,False,0.021149,0.002348,0.006428,0.025214,0.000083,Willy Santantines,True


In [3]:
data["cabin_1"] = " "
data["cabin_2"] = " "
data["cabin_3"] = " "

for n in range(len(data["Cabin"].values)):
    if type(data["Cabin"][n]) == float:
        data["cabin_1"][n] = "n"
        data["cabin_2"][n] = "n"
        data["cabin_3"][n] = "n"
    else:
        chars = [char for char in data["Cabin"][n]]
        data["cabin_1"][n] = chars[0]
        data["cabin_2"][n] = chars[2]
        data["cabin_3"][n] = chars[4]

data = data.drop(columns=["Cabin", "Name"])

features_encoder = OrdinalEncoder()
features_encoder.fit(data[["HomePlanet", "CryoSleep", "Destination", "VIP", "cabin_1", "cabin_2", "cabin_3"]])
data[["HomePlanet", "CryoSleep", "Destination", "VIP", "cabin_1", "cabin_2", "cabin_3"]] = features_encoder.transform(data[["HomePlanet", "CryoSleep", "Destination", "VIP", "cabin_1", "cabin_2", "cabin_3"]])

labels_encoder = OrdinalEncoder()
labels_encoder.fit(data[["Transported"]])
data[["Transported"]] = labels_encoder.transform(data[["Transported"]])

dump(features_encoder, open('features_encoder.pkl', 'wb'))

data = data.fillna(-1)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: Set

In [4]:
Y = data.pop("Transported")
X = data.drop(columns="PassengerId")

In [5]:
model = Sequential()
model.add(Dense(13, input_dim=13, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
# check if the model doesn't overfit first then for the final submission don't use validations
#model.fit(X, Y, epochs=50, batch_size=2, validation_split = 0.1)
model.fit(X, Y, epochs=30, batch_size=32)
_, accuracy = model.evaluate(X, Y)
print(f"accuracy is {accuracy}")

2022-05-15 08:04:28.351688: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-15 08:04:28.461123: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-15 08:04:28.462044: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-15 08:04:28.463253: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Epoch 1/30
272/272 [==============================] - 2s 2ms/step - loss: 0.2244 - accuracy: 0.6312
Epoch 2/30
272/272 [==============================] - 1s 2ms/step - loss: 0.1977 - accuracy: 0.6976
Epoch 3/30
272/272 [==============================] - 1s 2ms/step - loss: 0.1923 - accuracy: 0.7130
Epoch 4/30
272/272 [==============================] - 1s 2ms/step - loss: 0.1903 - accuracy: 0.7184
Epoch 5/30
272/272 [==============================] - 1s 3ms/step - loss: 0.1886 - accuracy: 0.7180
Epoch 6/30
272/272 [==============================] - 1s 3ms/step - loss: 0.1893 - accuracy: 0.7166
Epoch 7/30
272/272 [==============================] - 1s 3ms/step - loss: 0.1870 - accuracy: 0.7214
Epoch 8/30
272/272 [==============================] - 1s 4ms/step - loss: 0.1872 - accuracy: 0.7197
Epoch 9/30
272/272 [==============================] - 1s 2ms/step - loss: 0.1850 - accuracy: 0.7224
Epoch 10/30
272/272 [==============================] - 1s 2ms/step - loss: 0.1844 - accuracy: 0.7256

In [6]:
prediction_data = pd.read_csv("../input/spaceship-titanic/test.csv")

prediction_data[["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]] = min_max_scaler.transform(prediction_data[["Age", "RoomService", "FoodCourt", "ShoppingMall", "Spa", "VRDeck"]])   

prediction_data["cabin_1"] = " "
prediction_data["cabin_2"] = " "
prediction_data["cabin_3"] = " "

for n in range(len(prediction_data["Cabin"].values)):
    if type(prediction_data["Cabin"][n]) == float:
        prediction_data["cabin_1"][n] = "n"
        prediction_data["cabin_2"][n] = "n"
        prediction_data["cabin_3"][n] = "n"
    else:
        chars = [char for char in prediction_data["Cabin"][n]]
        prediction_data["cabin_1"][n] = chars[0]
        prediction_data["cabin_2"][n] = chars[2]
        prediction_data["cabin_3"][n] = chars[4]

prediction_data = prediction_data.drop(columns=["Cabin", "Name"])

prediction_data[["HomePlanet", "CryoSleep", "Destination", "VIP", "cabin_1", "cabin_2", "cabin_3"]] = features_encoder.transform(prediction_data[["HomePlanet", "CryoSleep", "Destination", "VIP", "cabin_1", "cabin_2", "cabin_3"]])

prediction_data = prediction_data.fillna(-1)

labels = prediction_data.pop("PassengerId")

predictions = model.predict(prediction_data)

my_submission = pd.DataFrame({
    "PassengerId": ["placeholder" for n in range(len(predictions))],
    "Transported": ["placeholder" for n in range(len(predictions))]
})

for n in range(len(predictions)):
    my_submission["PassengerId"][n] = labels[n]
    if predictions[n] > 0.5:
        my_submission["Transported"][n] = True
    else:
        my_submission["Transported"][n] = False

my_submission.to_csv('submission.csv', index=False)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarn